# Snow & Rain CNN
###     
###  Captain
### Snow Rain train model, validation and overfitting



## PREPARE
####          
#### Train Snow Images:  1200
#### Train Rain Images:  1200
####  Test Snow Images:   500
####  Test Rain Images:   500

### IMPORT LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
os.getcwd()


### DRIVE  DIRs

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:

# For some reason Colab was crashing, so I delegated the data


train_dir_m = "/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/train"
test_dir_m = "/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/test"

train_dir_predict = "/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/train"
test_dir_predict = "content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/test"

test_dir="/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN"
train_dir="/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN"


train_dir_snow=("/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/train/snowtr")
train_snow_fnames = os.listdir( train_dir_snow )
print(train_snow_fnames[:5])

train_dir_rain=("/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/train/raintr")
train_rain_fnames = os.listdir( train_dir_rain)
print(train_rain_fnames[:5])

test_dir_snowT=("/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/test/snowte")
test_snowT_fnames = os.listdir( test_dir_snowT )
print(test_snowT_fnames[:5])

test_dir_rainT=("/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/test/rainte")
test_rainT_fnames = os.listdir( test_dir_rainT)
print(test_rainT_fnames[:5])



## VISUALIZE SNOW & RAIN

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# 4 X 4
nrows = 4
ncols = 4

# index of images
pic_index = 0 
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=4

next_snow_pix = [os.path.join(train_dir_snow, fname) 
                for fname in train_snow_fnames[ pic_index-4:pic_index] 
               ]

next_rain_pix = [os.path.join(train_dir_rain, fname) 
                for fname in train_rain_fnames[ pic_index-4:pic_index]
               ]
# off grids
for i, img_path in enumerate(next_snow_pix+next_rain_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') 

  img = mpimg.imread(img_path)
  plt.imshow(img)

## CREATING MODELS 

### Add layers MODEL

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()

In [ ]:
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

####  Train model also use binary_crossentropy for loss and activate with sigmoid.

In [ ]:

from keras import optimizers
model.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=1e-4), 
                metrics=['acc'])

In [ ]:
# Use ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)



# train
train_generator = train_datagen.flow_from_directory(
    train_dir_m,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

# validate
validation_generator = test_datagen.flow_from_directory(
    test_dir_m,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=10,
    epochs=6,
    validation_data=validation_generator,
    validation_steps=20)

###  ACCURACY and LOSS

In [ ]:



accuracy=history.history["acc"]
loss=history.history['loss']
val_accuracy=history.history['val_acc']
val_loss=history.history['val_loss']
epochs=range(len(accuracy))

In [ ]:
plt.plot(epochs,accuracy)
plt.plot(epochs,val_accuracy)
plt.title("training and validation Accuracy")
plt.show()

In [ ]:
plt.plot(epochs,loss)
plt.plot(epochs,val_loss)
plt.title("training and Validation loss")
plt.show()

## PREDICTIONS

### IMAGE GENERATOR

In [ ]:


Batch_size = 32
image_shape = 150



In [ ]:
train_image_generator = ImageDataGenerator(
    # image augmentation
    rotation_range = 0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip=True,
    rescale = 1./255 #normalize the image 
             )
test_image_generator = ImageDataGenerator(
    rescale = 1./255
             )


train_image_gen = train_image_generator.flow_from_directory(
batch_size = Batch_size,
directory = train_dir_predict,
shuffle = True,
target_size = (image_shape,image_shape),
class_mode= "binary")


test_image_gen =test_image_generator.flow_from_directory(
batch_size = Batch_size,
directory = test_dir,
shuffle = False,
target_size = (image_shape,image_shape),
class_mode= "binary")



In [ ]:
train_image_gen.class_indices

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
image = load_img("/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/18.jpg", target_size=(image_shape, image_shape)) # load image and resize
type(image)

In [ ]:
img = img_to_array(image)
type(img)

In [ ]:
print(img.shape)

In [ ]:
# Dimension & Normalize

img = np.expand_dims(img, 0)
img = img.astype(np.float32)/255.0
print(img.shape)

### PREDICT MODEL

In [ ]:
model.predict(img)

In [ ]:
img0 = load_img("/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/2.jpg",target_size=(image_shape,image_shape))
img0 = img_to_array(img0)
img0 = np.expand_dims(img0,0)
img0 = img0.astype(np.float32)/255.0
model.predict(img0)

### DEFs

In [ ]:
def load_images(folder):
    outputs = []
    for f in os.listdir(folder):
        print(f)
        img = load_img(os.path.join(folder, f),target_size=(image_shape,image_shape))
        img = img_to_array(img)
        img = np.expand_dims(img,0)
        img = img.astype(np.float32)/255.0
        outputs.append(img)
    return np.concatenate(outputs)
  

def predict_classes(model, images, classes):
    output = []
    prediction = model.predict(images)

    class_label = {value:key for key, value in classes.items()}
    for p in prediction:
        if p> 0.5:
            output.append(class_label[1])
        else:
            output.append(class_label[0])
    return output


###  PLOT IMAGE, Rain or Snow

In [ ]:
# result predict
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
img = load_img('/content/gdrive/MyDrive/Colab Notebooks/KTGCOLAB/SNOWRAIN/5.jpg', target_size=(150, 150))
plt.imshow(img)
img = img_to_array(img)
img = img.reshape(1, 150, 150 , 3)
img = img.astype('float32')
result = model.predict(img)
if(result==1):
    print('Snow')
else:
    print("Rain")
plt.show()